# Lab 5: XCom và Data Sharing - Chia sẻ Dữ liệu giữa Tasks

## 🎯 Objectives
- Hiểu XCom (Cross-Communication) trong Airflow
- Sử dụng task return values để chia sẻ data
- XCom push và pull operations
- XCom với Task SDK (@task decorator)
- XCom với Operators (PythonOperator)
- Best practices cho data sharing
- XCom limitations và alternatives

## 📋 Prerequisites
- Hoàn thành Lab 1-4
- Hiểu task dependencies
- Airflow cluster đang chạy

## 🏗️ XCom Overview
XCom (Cross-Communication) là cơ chế của Airflow để chia sẻ dữ liệu giữa tasks:
- **XCom Push**: Lưu data vào XCom
- **XCom Pull**: Lấy data từ XCom
- **Automatic**: Task return values tự động được push vào XCom
- **Manual**: Sử dụng `xcom_push()` và `xcom_pull()` methods


## 1. Import Libraries và Setup


In [ ]:
# Import Airflow XCom và related modules
from airflow.sdk import DAG, task
from airflow.providers.standard.operators.python import PythonOperator
from airflow.providers.standard.operators.bash import BashOperator

import pendulum
from datetime import datetime
import json

print("✅ Airflow XCom modules imported successfully!")


## 2. XCom với Task SDK (@task decorator) - Automatic Return Values

Với Task SDK, return values tự động được push vào XCom. Đây là cách đơn giản nhất để chia sẻ data.


In [ ]:
# DAG với Task SDK - Automatic XCom
@dag(
    dag_id="xcom_task_sdk_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["xcom", "task-sdk"],
)
def xcom_task_sdk_dag():
    """
    ### XCom với Task SDK
    Task SDK tự động push return values vào XCom.
    """
    
    @task
    def extract_data():
        """Extract data và return - tự động push vào XCom"""
        data = {
            "users": [
                {"id": 1, "name": "Alice", "age": 30},
                {"id": 2, "name": "Bob", "age": 25},
                {"id": 3, "name": "Charlie", "age": 35},
            ],
            "total": 3,
            "timestamp": datetime.now().isoformat(),
        }
        print(f"Extracted {data['total']} users")
        return data  # Tự động push vào XCom
    
    @task
    def transform_data(data: dict):
        """Transform data - nhận từ XCom tự động"""
        users = data["users"]
        
        # Calculate statistics
        total_age = sum(user["age"] for user in users)
        avg_age = total_age / len(users)
        
        transformed = {
            "total_users": len(users),
            "average_age": avg_age,
            "max_age": max(user["age"] for user in users),
            "min_age": min(user["age"] for user in users),
        }
        print(f"Transformed data: {transformed}")
        return transformed  # Tự động push vào XCom
    
    @task
    def load_data(stats: dict):
        """Load data - nhận từ XCom tự động"""
        print(f"Loading statistics:")
        print(f"  Total users: {stats['total_users']}")
        print(f"  Average age: {stats['average_age']:.2f}")
        print(f"  Age range: {stats['min_age']} - {stats['max_age']}")
        return f"Loaded {stats['total_users']} users successfully"
    
    # Define workflow - data tự động pass qua XCom
    extracted = extract_data()
    transformed = transform_data(extracted)  # extracted tự động được pull từ XCom
    load_data(transformed)  # transformed tự động được pull từ XCom

# Create DAG
xcom_task_sdk_dag_instance = xcom_task_sdk_dag()

print("✅ XCom Task SDK DAG created!")
print(f"Tasks: {[task.task_id for task in xcom_task_sdk_dag_instance.tasks]}")
print("\n💡 Với Task SDK:")
print("  - Return values tự động push vào XCom")
print("  - Function parameters tự động pull từ XCom")
print("  - Không cần manual xcom_push/xcom_pull")


## 3. XCom với PythonOperator - Manual Push/Pull

Với PythonOperator, bạn cần manually push và pull XCom values sử dụng `xcom_push()` và `xcom_pull()`.


In [ ]:
# DAG với PythonOperator - Manual XCom
@dag(
    dag_id="xcom_python_operator_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["xcom", "python-operator"],
)
def xcom_python_operator_dag():
    """
    ### XCom với PythonOperator
    Manual push và pull XCom values với PythonOperator.
    """
    
    def extract_data(**context):
        """Extract data và push vào XCom manually"""
        data = {
            "records": [1, 2, 3, 4, 5],
            "sum": 15,
        }
        
        # Manual XCom push
        context['ti'].xcom_push(key='extracted_data', value=data)
        print(f"Pushed data to XCom: {data}")
        return "Extraction completed"
    
    def transform_data(**context):
        """Transform data - pull từ XCom manually"""
        # Manual XCom pull
        data = context['ti'].xcom_pull(key='extracted_data', task_ids='extract_data')
        
        if data:
            # Transform data
            transformed = {
                "total_records": len(data['records']),
                "sum": data['sum'],
                "average": data['sum'] / len(data['records']),
            }
            
            # Push transformed data
            context['ti'].xcom_push(key='transformed_data', value=transformed)
            print(f"Transformed data: {transformed}")
            return "Transformation completed"
        else:
            raise ValueError("No data found in XCom")
    
    def load_data(**context):
        """Load data - pull từ XCom manually"""
        # Pull từ task khác
        transformed = context['ti'].xcom_pull(key='transformed_data', task_ids='transform_data')
        
        if transformed:
            print(f"Loading data:")
            print(f"  Total records: {transformed['total_records']}")
            print(f"  Sum: {transformed['sum']}")
            print(f"  Average: {transformed['average']:.2f}")
            return "Load completed"
        else:
            raise ValueError("No transformed data found")
    
    # Tasks với PythonOperator
    extract_task = PythonOperator(
        task_id="extract_data",
        python_callable=extract_data,
    )
    
    transform_task = PythonOperator(
        task_id="transform_data",
        python_callable=transform_data,
    )
    
    load_task = PythonOperator(
        task_id="load_data",
        python_callable=load_data,
    )
    
    # Define dependencies
    extract_task >> transform_task >> load_task

# Create DAG
xcom_python_operator_dag_instance = xcom_python_operator_dag()

print("✅ XCom PythonOperator DAG created!")
print(f"Tasks: {[task.task_id for task in xcom_python_operator_dag_instance.tasks]}")
print("\n💡 Với PythonOperator:")
print("  - Sử dụng context['ti'].xcom_push() để push")
print("  - Sử dụng context['ti'].xcom_pull() để pull")
print("  - Cần specify key và task_ids")


## 4. XCom với Multiple Return Values

Tasks có thể return multiple values hoặc dictionaries, và chúng sẽ được tự động push vào XCom.


In [ ]:
# DAG với Multiple Return Values
@dag(
    dag_id="xcom_multiple_values_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["xcom", "multiple-values"],
)
def xcom_multiple_values_dag():
    """
    ### XCom với Multiple Return Values
    Tasks có thể return dictionaries với multiple values.
    """
    
    @task(multiple_outputs=True)  # Enable multiple outputs
    def extract_multiple_sources():
        """Extract từ nhiều sources và return dictionary"""
        source_a = {"records": [1, 2, 3], "source": "A"}
        source_b = {"records": [4, 5, 6], "source": "B"}
        
        return {
            "source_a": source_a,
            "source_b": source_b,
            "total_records": 6,
        }
    
    @task
    def process_source_a(source_a: dict):
        """Process source A"""
        print(f"Processing {source_a['source']}: {source_a['records']}")
        return sum(source_a['records'])
    
    @task
    def process_source_b(source_b: dict):
        """Process source B"""
        print(f"Processing {source_b['source']}: {source_b['records']}")
        return sum(source_b['records'])
    
    @task
    def aggregate_results(result_a: int, result_b: int):
        """Aggregate results từ cả 2 sources"""
        total = result_a + result_b
        print(f"Aggregated result: {total}")
        return total
    
    # Extract data với multiple outputs
    extracted = extract_multiple_sources()
    
    # Access individual values từ dictionary
    process_a = process_source_a(extracted['source_a'])
    process_b = process_source_b(extracted['source_b'])
    
    # Aggregate
    aggregate_results(process_a, process_b)

# Create DAG
xcom_multiple_values_dag_instance = xcom_multiple_values_dag()

print("✅ XCom Multiple Values DAG created!")
print(f"Tasks: {[task.task_id for task in xcom_multiple_values_dag_instance.tasks]}")
print("\n💡 Multiple outputs:")
print("  - Sử dụng @task(multiple_outputs=True)")
print("  - Return dictionary với multiple keys")
print("  - Access values bằng key: extracted['source_a']")


## 5. XCom với Lists và Complex Data Structures

XCom có thể lưu trữ lists, dictionaries, và các complex data structures (nhưng có size limits).


In [ ]:
# DAG với Complex Data Structures
@dag(
    dag_id="xcom_complex_data_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["xcom", "complex-data"],
)
def xcom_complex_data_dag():
    """
    ### XCom với Complex Data Structures
    XCom có thể lưu lists, dictionaries, và nested structures.
    """
    
    @task
    def generate_complex_data():
        """Generate complex nested data structure"""
        data = {
            "metadata": {
                "timestamp": datetime.now().isoformat(),
                "version": "1.0",
            },
            "users": [
                {
                    "id": 1,
                    "name": "Alice",
                    "scores": [95, 87, 92],
                    "metadata": {"department": "Engineering"}
                },
                {
                    "id": 2,
                    "name": "Bob",
                    "scores": [78, 85, 90],
                    "metadata": {"department": "Sales"}
                },
            ],
            "statistics": {
                "total_users": 2,
                "average_score": 88.5,
            }
        }
        print(f"Generated complex data with {data['statistics']['total_users']} users")
        return data
    
    @task
    def process_users(complex_data: dict):
        """Process users từ complex data"""
        users = complex_data['users']
        
        results = []
        for user in users:
            avg_score = sum(user['scores']) / len(user['scores'])
            results.append({
                "user_id": user['id'],
                "name": user['name'],
                "average_score": avg_score,
                "department": user['metadata']['department']
            })
        
        print(f"Processed {len(results)} users")
        return results
    
    @task
    def generate_report(user_results: list, metadata: dict):
        """Generate report từ processed data"""
        print("=" * 60)
        print("User Performance Report")
        print("=" * 60)
        print(f"Generated at: {metadata['timestamp']}")
        print(f"Version: {metadata['version']}")
        print("\nUser Details:")
        for result in user_results:
            print(f"  {result['name']} ({result['department']}): {result['average_score']:.2f}")
        print("=" * 60)
        return "Report generated"
    
    # Extract complex data
    complex_data = generate_complex_data()
    
    # Process users
    user_results = process_users(complex_data)
    
    # Generate report với multiple inputs
    generate_report(user_results, complex_data['metadata'])

# Create DAG
xcom_complex_data_dag_instance = xcom_complex_data_dag()

print("✅ XCom Complex Data DAG created!")
print(f"Tasks: {[task.task_id for task in xcom_complex_data_dag_instance.tasks]}")
print("\n💡 Complex data structures:")
print("  - XCom hỗ trợ nested dictionaries và lists")
print("  - Có thể access nested values: data['metadata']['timestamp']")
print("  - Lưu ý: XCom có size limits (default: 48KB)")


## 6. XCom Pull từ Multiple Tasks

Một task có thể pull XCom values từ nhiều upstream tasks khác nhau.


In [ ]:
# DAG với XCom từ Multiple Tasks
@dag(
    dag_id="xcom_multiple_tasks_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["xcom", "multiple-tasks"],
)
def xcom_multiple_tasks_dag():
    """
    ### XCom từ Multiple Tasks
    Pull XCom values từ nhiều upstream tasks.
    """
    
    @task
    def extract_source_a():
        """Extract từ source A"""
        data = {"source": "A", "records": [1, 2, 3], "sum": 6}
        print(f"Extracted from source A: {data}")
        return data
    
    @task
    def extract_source_b():
        """Extract từ source B"""
        data = {"source": "B", "records": [4, 5, 6], "sum": 15}
        print(f"Extracted from source B: {data}")
        return data
    
    @task
    def extract_source_c():
        """Extract từ source C"""
        data = {"source": "C", "records": [7, 8, 9], "sum": 24}
        print(f"Extracted from source C: {data}")
        return data
    
    @task
    def merge_data(source_a: dict, source_b: dict, source_c: dict):
        """Merge data từ cả 3 sources"""
        all_records = (
            source_a['records'] + 
            source_b['records'] + 
            source_c['records']
        )
        total_sum = source_a['sum'] + source_b['sum'] + source_c['sum']
        
        merged = {
            "all_records": all_records,
            "total_sum": total_sum,
            "total_records": len(all_records),
            "sources": [source_a['source'], source_b['source'], source_c['source']]
        }
        
        print(f"Merged data: {merged}")
        return merged
    
    @task
    def finalize(merged_data: dict):
        """Finalize với merged data"""
        print(f"Finalizing with {merged_data['total_records']} records")
        print(f"Total sum: {merged_data['total_sum']}")
        print(f"Sources: {', '.join(merged_data['sources'])}")
        return "Finalized"
    
    # Extract từ nhiều sources (parallel)
    source_a_data = extract_source_a()
    source_b_data = extract_source_b()
    source_c_data = extract_source_c()
    
    # Merge data từ cả 3 sources
    merged = merge_data(source_a_data, source_b_data, source_c_data)
    
    # Finalize
    finalize(merged)

# Create DAG
xcom_multiple_tasks_dag_instance = xcom_multiple_tasks_dag()

print("✅ XCom Multiple Tasks DAG created!")
print(f"Tasks: {[task.task_id for task in xcom_multiple_tasks_dag_instance.tasks]}")
print("\n💡 Multiple upstream tasks:")
print("  - Task có thể nhận inputs từ nhiều upstream tasks")
print("  - Function parameters map với return values từ upstream tasks")
print("  - Tất cả upstream tasks phải complete trước khi task này chạy")


In [ ]:
# DAG minh họa XCom Best Practices
@dag(
    dag_id="xcom_best_practices_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["xcom", "best-practices"],
)
def xcom_best_practices_dag():
    """
    ### XCom Best Practices Example
    Minh họa best practices khi sử dụng XCom.
    """
    
    @task
    def extract_metadata_only():
        """
        ✅ Best Practice: Chỉ pass metadata, không pass large data
        Thay vì pass toàn bộ data, chỉ pass file path hoặc reference
        """
        # Simulate: Thay vì pass large dataset, chỉ pass file path
        file_path = "/tmp/data/large_dataset.parquet"
        metadata = {
            "file_path": file_path,
            "record_count": 1000000,
            "file_size_mb": 250,
            "schema": ["id", "name", "value"],
        }
        print(f"Extracted metadata: {metadata}")
        return metadata  # Chỉ pass metadata, không pass data
    
    @task
    def process_file(metadata: dict):
        """
        ✅ Best Practice: Process file từ path, không từ XCom
        """
        file_path = metadata['file_path']
        print(f"Processing file: {file_path}")
        print(f"Records: {metadata['record_count']}")
        # In thực tế, đọc file từ path và process
        return {"status": "processed", "records_processed": metadata['record_count']}
    
    @task
    def store_summary(summary: dict):
        """
        ✅ Best Practice: Chỉ store summary/aggregated data
        """
        print(f"Storing summary: {summary}")
        return "Summary stored"
    
    # Workflow với best practices
    metadata = extract_metadata_only()
    summary = process_file(metadata)
    store_summary(summary)

# Create DAG
xcom_best_practices_dag_instance = xcom_best_practices_dag()

print("✅ XCom Best Practices DAG created!")
print("\n📋 XCom Best Practices:")
print("=" * 60)
print("✅ DO:")
print("  - Chỉ pass small data (< 48KB)")
print("  - Pass metadata/references thay vì large datasets")
print("  - Pass file paths thay vì file contents")
print("  - Pass aggregated/summary data")
print("  - Sử dụng Task SDK cho automatic XCom")
print("\n❌ DON'T:")
print("  - Pass large datasets qua XCom")
print("  - Pass binary data qua XCom")
print("  - Pass sensitive data (use Variables/Connections)")
print("  - Rely on XCom cho data storage")
print("=" * 60)
print("\n💡 Alternatives cho Large Data:")
print("  - File storage (S3, GCS, local files)")
print("  - Databases")
print("  - External storage systems")
print("  - Pass only references/IDs qua XCom")


## 8. XCom với Task Mapping và Dynamic Tasks

XCom hoạt động tốt với dynamic task mapping - mỗi mapped task instance có XCom riêng.


In [ ]:
# DAG với XCom và Task Mapping
@dag(
    dag_id="xcom_task_mapping_example",
    schedule=None,
    start_date=pendulum.datetime(2024, 1, 1, tz="UTC"),
    catchup=False,
    tags=["xcom", "task-mapping"],
)
def xcom_task_mapping_dag():
    """
    ### XCom với Task Mapping
    XCom hoạt động với dynamic task mapping.
    """
    
    @task
    def get_files_to_process():
        """Get list of files to process"""
        files = [
            {"path": "/data/file1.csv", "size": 1000},
            {"path": "/data/file2.csv", "size": 2000},
            {"path": "/data/file3.csv", "size": 1500},
        ]
        print(f"Found {len(files)} files to process")
        return files
    
    @task
    def process_file(file_info: dict):
        """Process một file - sẽ được map cho mỗi file"""
        file_path = file_info['path']
        file_size = file_info['size']
        
        # Simulate processing
        records_processed = file_size // 100
        
        result = {
            "file_path": file_path,
            "records_processed": records_processed,
            "status": "success"
        }
        
        print(f"Processed {file_path}: {records_processed} records")
        return result
    
    @task
    def aggregate_results(results: list):
        """Aggregate results từ tất cả mapped tasks"""
        total_records = sum(r['records_processed'] for r in results)
        total_files = len(results)
        
        summary = {
            "total_files": total_files,
            "total_records": total_records,
            "average_records_per_file": total_records / total_files if total_files > 0 else 0
        }
        
        print(f"Aggregated summary: {summary}")
        return summary
    
    # Get files
    files = get_files_to_process()
    
    # Process files với dynamic mapping
    # Mỗi mapped task instance sẽ có XCom riêng
    processed_files = process_file.expand(file_info=files)
    
    # Aggregate - nhận list của tất cả results
    aggregate_results(processed_files)

# Create DAG
xcom_task_mapping_dag_instance = xcom_task_mapping_dag()

print("✅ XCom Task Mapping DAG created!")
print(f"Tasks: {[task.task_id for task in xcom_task_mapping_dag_instance.tasks]}")
print("\n💡 XCom với Task Mapping:")
print("  - Mỗi mapped task instance có XCom riêng")
print("  - Aggregate task nhận list của tất cả results")
print("  - XCom key tự động include map index")


## 9. Tóm tắt và Next Steps

### ✅ Những gì đã học:
1. XCom basics - Cross-communication giữa tasks
2. Task SDK automatic XCom - Return values tự động
3. PythonOperator manual XCom - Push/pull operations
4. Multiple return values với dictionaries
5. Complex data structures trong XCom
6. Pull từ multiple upstream tasks
7. XCom limitations và best practices
8. XCom với task mapping

### 📚 Next Lab:
- **Lab 6**: Scheduling và Timetables
- Cron expressions
- Timedelta schedules
- Custom timetables
- Catchup và data intervals

### 🔗 Useful Links:
- [XCom Documentation](https://airflow.apache.org/docs/apache-airflow/3.1.1/core-concepts/xcoms.html)
- [Task SDK XCom](https://airflow.apache.org/docs/apache-airflow/3.1.1/task-sdk/index.html)
- [XCom Best Practices](https://airflow.apache.org/docs/apache-airflow/3.1.1/best-practices.html#xcom)

### 💡 Key Takeaways:

**XCom Size Limits:**
- Default: 48KB per value
- Configurable via `xcom_max_value_size`
- Không nên pass large data

**Best Practices:**
- ✅ Pass metadata/references
- ✅ Pass file paths thay vì contents
- ✅ Use Task SDK cho automatic XCom
- ❌ Don't pass large datasets
- ❌ Don't use XCom như database

**Alternatives:**
- File storage (S3, GCS, local)
- Databases
- External APIs
- Airflow Variables (cho config)

### 💡 Exercises:
1. Tạo DAG với XCom passing metadata giữa tasks
2. Implement data pipeline với file paths qua XCom
3. Sử dụng multiple return values
4. Combine XCom với task mapping
5. Implement error handling với XCom
